<a href="https://colab.research.google.com/github/rakeshreddyd56/BusinessAnalystProject/blob/main/Intelligent_Customer_Segmentation_for_Personalized_Marketing_Campaigns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/customers-data - customers-data.csv')

In [3]:
df.head()

,customer_id,products_purchased,complains,money_spent
0,1000661,1,0,260.0
1,1001914,1,0,790.2
2,1002167,3,0,234.2
3,1002387,1,0,890.0
4,1002419,2,0,103.0


In [4]:
df.describe()

,customer_id,products_purchased,complains,money_spent
count,2.400000e+04,24000.000000,24000.000000,24000.000000
mean,1.001908e+06,1.600000,0.010000,456.830000
std,6.489202e+02,0.800017,0.117901,320.704042
min,1.000660e+06,1.000000,0.000000,102.000000
25%,1.001913e+06,1.000000,0.000000,233.700000
50%,1.002166e+06,1.000000,0.000000,260.500000
75%,1.002387e+06,2.000000,0.000000,793.200000
max,1.002419e+06,3.000000,2.000000,894.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id         24000 non-null  int64  
 1   products_purchased  24000 non-null  int64  
 2   complains           24000 non-null  int64  
 3   money_spent         24000 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 750.1 KB


In [6]:
df.shape

(24000, 4)

In [8]:
df.isna().sum()

customer_id           0
products_purchased    0
complains             0
money_spent           0
dtype: int64

No nulls, so we don't need to impute anything

###KMEANS

In [9]:
# Import required libraries

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [11]:
df1 = df.copy()

In [12]:
# Define K-means model
kmeans_model = KMeans(init='k-means++',  max_iter=400, random_state=42)

In [14]:
# Training the model
kmeans_model.fit(df1[['products_purchased','complains',
'money_spent']])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(max_iter=400, random_state=42)

In [15]:
def try_different_clusters(K, data):

    cluster_values = list(range(1, K+1))
    inertias=[]

    for c in cluster_values:
        model = KMeans(n_clusters = c,init='k-means++',max_iter=400,random_state=42)
        model.fit(data)
        inertias.append(model.inertia_)

    return inertias

In [17]:
outputs = try_different_clusters(12, df1[['products_purchased','complains','money_spent']])
distances = pd.DataFrame({"clusters": list(range(1, 13)),"sum of squared distances": outputs})

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [18]:
figure = go.Figure()
figure.add_trace(go.Scatter(x=distances["clusters"], y=distances["sum of squared distances"]))

figure.update_layout(xaxis = dict(tick0 = 1,dtick = 1,tickmode = 'linear'),
                  xaxis_title="Number of clusters",
                  yaxis_title="Sum of squared distances",
                  title_text="Finding optimal number of clusters using elbow method")
figure.show()

In [20]:
# Re-Train K means model with k=5
kmeans_model_new = KMeans(n_clusters = 5,init='k-means++',max_iter=400,random_state=42)

kmeans_model_new.fit_predict(df1[['products_purchased','complains','money_spent']])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



array([3, 1, 4, ..., 4, 2, 0], dtype=int32)

In [21]:
cluster_centers = kmeans_model_new.cluster_centers_
data = np.expm1(cluster_centers)
points = np.append(data, cluster_centers, axis=1)
points

<ipython-input-21-746358cf1b4d>:2: RuntimeWarning:

overflow encountered in expm1



array([[ 6.38905610e+000,  5.01252086e-003,  1.14301649e+045,
         2.00000000e+000,  5.00000000e-003,  1.03750000e+002],
       [ 1.71828183e+000,  1.00501671e-002,              inf,
         1.00000000e+000,  1.00000000e-002,  7.92700000e+002],
       [ 1.71828183e+000,  2.02013400e-002,              inf,
         1.00000000e+000,  2.00000000e-002,  8.91750000e+002],
       [ 1.71828183e+000,  1.51130646e-002,  2.24315755e+113,
         1.00000000e+000,  1.50000000e-002,  2.61000000e+002],
       [ 1.90855369e+001, -3.97251676e-016,  1.09015568e+102,
         3.00000000e+000, -3.97251676e-016,  2.34950000e+002]])

In [22]:
points = np.append(points, [[0], [1], [2], [3], [4]], axis=1)
df1["clusters"] = kmeans_model_new.labels_

In [24]:
df1.head()

,customer_id,products_purchased,complains,money_spent,clusters
0,1000661,1,0,260.0,3
1,1001914,1,0,790.2,1
2,1002167,3,0,234.2,4
3,1002387,1,0,890.0,2
4,1002419,2,0,103.0,0


###Visualisation

In [26]:
# visualize clusters
figure = px.scatter_3d(df1,
                    color='clusters',
                    x="products_purchased",
                    y="complains",
                    z="money_spent",
                    category_orders = {"clusters": ["0", "1", "2", "3", "4"]}
                    )
figure.update_layout()
figure.show()